# dataset

In [ ]:
import pandas as pd
train=pd.read_csv("/content/drive/MyDrive/NLP/Copy of train.csv")
test=pd.read_csv("/content/drive/MyDrive/NLP/Copy of test.csv")
labels=pd.read_csv("/content/drive/MyDrive/NLP/Copy of labels.csv")


In [ ]:
train_action=train[train['Action']==1]
train_set=train_action.sample(100).script.values
clean_train_set=[]
for i in range(len(train_set)):
    st=str(train_set[i].replace('\n', '').replace('\t',''))
    st=' '.join(st.split())
    clean_train_set.append(st)

In [ ]:
test_action=test[test['Action']==1]
test_set=test_action.sample(50).script.values

clean_test_set=[]
for i in range(len(test_set)):
    st=str(test_set[i].replace('\n', '').replace('\t',''))
    st=' '.join(st.split())
    clean_test_set.append(st)


In [ ]:
with open('/content/drive/MyDrive/NLP/train.txt','w') as f:
    
    f.writelines(clean_train_set)

In [ ]:
with open('/content/drive/MyDrive/NLP/test.txt','w') as f:
    
    f.writelines(clean_test_set)

# LSTM based model

In [1]:
import tensorflow as tf
import numpy as np
import os
import shutil
path_to_file = "/content/drive/MyDrive/NLP/train.txt"
text = open(path_to_file, 'r').read()
print('Length of Character {}'.format(len(text)))
print(text[:250])
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)
# Create a mapping from characters to numbers and vice versa
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
print("{} is mapped to {}".format(text[:10], text_as_int[:10]))
# Maximum sentence we are inputing to the RNN
seq_length =100
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)
# Creating dataset
# from_tensor_slices is like creating a generator for our dataset and is suitable for handling
# large datasets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# .take is like iloc in pandas
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

def split_input_target(chunk):
    """
    This function generate input and target text from the given text.
    Input text does not contain last part and target doesnot contain first character
    """
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset
#Constants for model
vocab_size = len(vocab)
embedding_dim = 512
# Using half of rnn_units for LSTM
# Speed of training was reduced to half, so i can try 1024 units
rnn_units = 1024
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

#Here we are chosing the next character randomly based on its probablity
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
#idk what this line does
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices
#Decoding what this means
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 10 #takes a long time
#loss: 0.8658
#epochs should be at least 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

#tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.save('Model_3_LSTM0.h5')
def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
model.save('Model_3_LSTM.h5')

Length of Character 12648575
GET CARTER Screenplay by Mike Hodges Based on the novel Jack's Return Home by Ted Lewis Revised Draft, 1971 Copyright (c) 1971 Turner Entertainment Co All Rights Reserved. Copyright (c) 1971 Mike Hodges All Rights Reserved. FADE IN: EXT. PENTHOUSE AP
137 unique characters
[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\x80', '\x82', '\x83', '\x8c', '\x90', '\x91', '\x92', '\x93', '\x96', '\x97', '\x98', '\x99', '\x9c', '\x9d', '\x9e', '¢', '£', '¦', '©', '¬', '\xad', '°', '³', '´', '·', 'º', '»', '½', '¿', 'Â', 'Ã', 'Ç', 'È', 'É', 'Ï', 'Ñ', 'Õ', 'Ú'

Spock the man with the trees and the rifle sits behind his door. GABE from the back of the hallway the face he was once she disappears. He gets the harness off the street and runs into a stop laughing and removes the hot stream of his feet and are staring at the others. Ashe notices somebody sauntering the screen, but it's like a pack. JAMES looks at it. He clicks the machine gun away. He stares back at her clean. He rushes into the sky above her head- face. JUDAH What are you doing in here? INDY (cool) You want the idea of head. ASHE I don't know what you need to do, sir. And I don't want to take care of them... JAMES can only reach the track, but no saying that. She takes the presence of the crew as he responds in the moments of the diner. They all look around at Lee. Maggie explaining a monitor. He just told me about the best fridge. Chinese ring of traffic -- which is unbare -- but a gurgling falling below, it hits the street, grabs the bed with a squad of his last. INSERT - COMMUN

In [2]:
print(generate_text(model, start_string="don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you."))

don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you. TASLIMA And you're the one needed to be on it, JAMES, 12-year-old boy... 100 yards from the world, the man he leads Edwards to the bed and take off running up to the left. The rocky glowing metallic car appears wildly, flipping the ring them. EXT. CRAWLER TWO of them have the rest of the city who looks better than the car panel is rough -- more above while street with a very stunned and ten degree flashing powers, with the galaxy. AUDREY He was truly away from Castor. BREAKENRIDGE The team is comin' up! INT. MEETING ROOM - CONTINUOUS - DAY 141 JAMES watches as the circle around her cheek is standing outside the tunnel. The chimpanzee part of a lid they can see the stunned dead body. It's empty -- JAMES What do you me

# Word-Level Text Generation with GPT-2



In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd 'drive/MyDrive/Colab Notebooks/nlg_tales_generation'

Mounted at /content/drive
[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/nlg_tales_generation'
/content


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.3 MB/s eta 0:00:00


In [3]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

In [4]:
train_path = 'data/train.txt'
test_path = 'data/test.txt'

## Text tokenization

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))

vocabulary size: 50257, max squence length: 1024
tokenize sequence "Once upon a time in a little village": {'input_ids': [7454, 2402, 257, 640, 287, 257, 1310, 7404], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Load dataset

In [8]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/drive/MyDrive/NLP/train.txt',
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/test.txt",
    block_size=128)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## Fine-tune model



In [10]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [13]:
training_args = TrainingArguments(
    output_dir = 'data/out', # the output directory for the model predictions and checkpoints
    overwrite_output_dir = True, # overwrite the content of the output directory
    per_device_train_batch_size = 32, # the batch size for training
    per_device_eval_batch_size = 32, # the batch size for evaluation
    learning_rate = 5e-5, # defaults to 5e-5
    num_train_epochs = 6, # total number of training epochs to perform
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer.train()              

***** Running training *****
  Num examples = 17280
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3240
  Number of trainable parameters = 124439808


Step,Training Loss
500,1.366400
1000,1.222500
1500,1.155200
2000,1.119200
2500,1.098700
3000,1.076400


Saving model checkpoint to data/out/checkpoint-500
Configuration saved in data/out/checkpoint-500/config.json
Model weights saved in data/out/checkpoint-500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1000
Configuration saved in data/out/checkpoint-1000/config.json
Model weights saved in data/out/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1500
Configuration saved in data/out/checkpoint-1500/config.json
Model weights saved in data/out/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-2000
Configuration saved in data/out/checkpoint-2000/config.json
Model weights saved in data/out/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-2500
Configuration saved in data/out/checkpoint-2500/config.json
Model weights saved in data/out/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-3000
Configuration saved in data/out/checkpoint-3000/config.json
M

TrainOutput(global_step=3240, training_loss=1.165084900090724, metrics={'train_runtime': 3543.1811, 'train_samples_per_second': 29.262, 'train_steps_per_second': 0.914, 'total_flos': 6772689469440000.0, 'train_loss': 1.165084900090724, 'epoch': 6.0})

In [15]:
trainer.save_model() 

Saving model checkpoint to data/out
Configuration saved in data/out/config.json
Model weights saved in data/out/pytorch_model.bin


## Text generation

In [16]:
generator = pipeline('text-generation', tokenizer='gpt2', model='data/out')

loading configuration file data/out/config.json
Model config GPT2Config {
  "_name_or_path": "data/out",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading configur

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_ran

In [17]:
print(generator('Once upon a time', max_length=40)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time,\n                            and a hundred generations to live


## Text generation with different decoding methods

In [18]:
text_beam = generator('Once upon a time',
                      max_length=40,
                      num_beams=5)
print(text_beam[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the\n                                


## random sampling

In [25]:
text_random_sampling = generator('in a cold night',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


in a cold night.\n\n                               


## top k sampling

In [39]:
text_k_sampling = generator("don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you.",
                            max_length=500,
                            top_k=40,
                            do_sample=True)
print(str(text_k_sampling[0]['generated_text']).replace(r"\n",'\n'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you.
           Now you have to answer to him!
                (beat)
              What I want is for you to join the army.

                        DULCEA
             Where do you find him?


                      LIZ
                      (beat)
          He seems to like the idea of becoming
            a priest.


      Dulcea\'s eyes close now. Hellboy stands there, confused.


                      DULCEA
                 (breathless)
              It wouldn\'t be my father; I\'ll always be his wife!
             (a beat)
        That\'s exactly why we\'re here,
               (sir)
         (to Liz)
         Your mother would do it the same for you.



                       L


## top p sampling

In [35]:
text_p_sampling = generator("don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you.",
                            max_length=500,
                            top_k=0,
                            top_p=0.92,
                            do_sample=True)
print(str(text_p_sampling[0]['generated_text']).replace(r"\n",'\n'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


don't know who you are. I don't know what you want. If you are looking for ransom I can tell you I don't have money, but what I do have are a very particular set of skills. Skills I have acquired over a very long career. Skills that make me a nightmare for people like you.

I want to be there with you.

She uses her right arm as an arms clasp, pulling him towards her.

He places a hand on her shoulder.

HIS POV

Colossus is on his back, waving the
arm with one arm. We see he is reaching into his chest as his heart starts to
carry out our information. The Chamber is filled with glowing blue
theater, flashing electrical lights and pulsating magnetic fields around it.

Ivan is half way through the maze. He looks past Arrakis, down to the
bridge. Diamond is above him -

CARETAKER

Our batteries are dangerously low. I need this man to
don\'t abandon us - or else he will attack us all
evidedly. We\'ll see if he can get out of there.

NEXT - INDUSTRIAL LIBRARY

A series of fire engines roll u